# WARNING

### Use deduplicator.ipynb to find and remove such duplicated files:

### cars_0001.jpg

### cars_0001.jpeg

In [1]:
import os
import yaml
import json
import glob
import shutil
from PIL import Image
from collections import defaultdict
from tqdm import tqdm

In [2]:
def yolo_to_coco(yolo_dir, output_json):
    print("Starting YOLO to COCO conversion...")
    
    # Load data.yaml to get class names
    data_yaml = os.path.join(yolo_dir, 'data.yaml')
    with open(data_yaml, 'r') as f:
        data = yaml.safe_load(f)
    class_names = data['names']
    
    # Build categories list
    categories = []
    for idx, name in enumerate(class_names):
        category = {
            'id': idx + 1,
            'name': name,
            'supercategory': ''
        }
        categories.append(category)

    # Initialize COCO JSON structure
    coco_json = {
        'licenses': [{'name': '', 'id': 0, 'url': ''}],
        'info': {'contributor': '', 'date_created': '', 'description': '', 'url': '', 'version': '', 'year': ''},
        'categories': categories,
        'images': [],
        'annotations': []
    }

    annotation_id = 1
    image_id = 1
    total_annotations = 0
    total_images = 0
    
    # Dictionary to keep track of file name occurrences
    filename_count = defaultdict(int)

    # Process images and labels
    splits = ['train', 'val', 'test']
    for split in splits:
        if split in data:
            print(f"\nProcessing {split} split...")
            images_dir = os.path.join(yolo_dir, data[split].replace('../', ''))
            labels_dir = images_dir.replace('images', 'labels')

            image_files = glob.glob(os.path.join(images_dir, '*'))
            for img_file in tqdm(image_files, desc=f"Processing {split} images"):
                base_filename = os.path.basename(img_file)
                
                # Create unique filename
                filename_count[base_filename] += 1
                if filename_count[base_filename] > 1:
                    name, ext = os.path.splitext(base_filename)
                    unique_filename = f"{name}_{filename_count[base_filename]}{ext}"
                else:
                    unique_filename = base_filename

                img = Image.open(img_file)
                width, height = img.size
                img_info = {
                    'id': image_id,
                    'width': width,
                    'height': height,
                    'file_name': unique_filename,
                    'license': 0,
                    'flickr_url': '',
                    'coco_url': '',
                    'date_captured': ''
                }
                coco_json['images'].append(img_info)
                total_images += 1

                # Corresponding label file
                label_file = os.path.join(labels_dir, os.path.splitext(os.path.basename(img_file))[0] + '.txt')
                if os.path.exists(label_file):
                    with open(label_file, 'r') as f:
                        lines = f.readlines()
                    for line in lines:
                        parts = line.strip().split()
                        if len(parts) >= 5:
                            class_id = int(parts[0])
                            bbox = [float(x) for x in parts[1:5]]

                            # Check if there are more points (segmentation)
                            if len(parts) > 5:
                                segmentation = [float(x) for x in parts[1:]]
                                segmentation_pixel = []
                                for idx, point in enumerate(segmentation):
                                    if idx % 2 == 0:
                                        segmentation_pixel.append(segmentation[idx] * width)
                                    else:
                                        segmentation_pixel.append(segmentation[idx] * height)
                            else:
                                x_center, y_center, w, h = bbox
                                x_center *= width
                                y_center *= height
                                w *= width
                                h *= height
                                x_min = x_center - w / 2
                                y_min = y_center - h / 2
                                segmentation_pixel = [
                                    x_min, y_min,
                                    x_min + w, y_min,
                                    x_min + w, y_min + h,
                                    x_min, y_min + h
                                ]

                            x_center, y_center, w, h = bbox[:4]
                            x_center *= width
                            y_center *= height
                            w *= width
                            h *= height
                            x_min = x_center - w / 2
                            y_min = y_center - h / 2

                            annotation = {
                                'id': annotation_id,
                                'image_id': image_id,
                                'category_id': class_id + 1,
                                'segmentation': [segmentation_pixel],
                                'area': w * h,
                                'bbox': [x_min, y_min, w, h],
                                'iscrowd': 0,
                                'attributes': {
                                    'Damage location': '',
                                    'Severity': '',
                                    'Recommendation': '',
                                    'occluded': False
                                }
                            }
                            coco_json['annotations'].append(annotation)
                            annotation_id += 1
                            total_annotations += 1
                image_id += 1

    # Save COCO JSON
    with open(output_json, 'w') as f:
        json.dump(coco_json, f)
    
    print("\nConversion completed successfully!")
    print(f"Total images processed: {total_images}")
    print(f"Total annotations created: {total_annotations}")
    print(f"Number of categories: {len(categories)}")
    print(f"Output saved to: {output_json}")

In [3]:
# Usage
yolo_directory = r'C:\Users\Sakib\Desktop\samples'
output_coco_json = 'instances_default.json'

yolo_to_coco(yolo_directory, output_coco_json)

Starting YOLO to COCO conversion...

Processing train split...


Processing train images: 100%|██████████| 1025/1025 [00:03<00:00, 266.14it/s]



Processing val split...


Processing val images: 100%|██████████| 82/82 [00:00<00:00, 445.20it/s]



Processing test split...


Processing test images: 0it [00:00, ?it/s]



Conversion completed successfully!
Total images processed: 1107
Total annotations created: 32566
Number of categories: 29
Output saved to: instances_default.json
